In [0]:
!pip install -q fake_useragent

In [0]:
from fake_useragent import UserAgent
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import requests

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Proyectos_Ley_Congreso/notebooks")

In [0]:
ua=UserAgent()
user_agent=ua.chrome

### Extraer links y los archivos html

In [0]:
start_url ="http://www2.congreso.gob.pe/Sicr/TraDocEstProc/CLProLey2016.nsf/Local%20Por%20Numero%20Inverso?OpenView"
url_base="http://www2.congreso.gob.pe"

In [0]:
sw=True
ind=1
list_links=[]
list_soup=[]
while sw==True:
  url=start_url+"&Start={}".format(ind)
  request = urllib.request.Request(url, headers = {"User-Agent":user_agent})
  page=urllib.request.urlopen(request)
  soup=BeautifulSoup(page, "html.parser")
  ind+=99
  try:
    if soup.find("h2").text=="No documents found":
      sw=False
  except:
    list_links.append(url)
    list_soup.append(soup)
    continue

### Obtener la metadata de los proyectos de ley

In [0]:
codigo = []
fechUlt = []
fechPres = []
estado = []
tituloProy = []
linkProy = []
for soup in list_soup:
  table = soup.find_all("table")[1]
  for i in table.find_all("tr")[1:]:
    codigo.append(i.find("a").text)
    fechUlt.append(i.find_all("td")[1].text)
    fechPres.append(i.find_all("td")[2].text)
    estado.append(i.find_all("td")[3].text)
    tituloProy.append(i.find_all("td")[4].text)
    linkProy.append(url_base+i.find("a").get("href"))

In [0]:
dataGeneral={"codigo":codigo,
      "fechUlt":fechUlt,
      "fechPres":fechPres,
      "estado":estado,
      "tituloProy":tituloProy,
      "linkProy":linkProy}
dataGeneral=pd.DataFrame(dataGeneral)
dataGeneral.drop_duplicates(inplace=True)

In [0]:
print(dataGeneral.shape)
dataGeneral.head()

(4849, 6)


,codigo,fechUlt,fechPres,estado,tituloProy,linkProy
0,04849/2019-CR,10/09/2019,09/30/2019,Decretado a...,"LEY DE REFORMA CONSTITUCIONAL, QUE ESTABLECE L...",http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
1,04848/2019-PE,10/10/2019,09/30/2019,En comisión,RESOLUCIÓN LEGISLATIVA QUE CONCEDE LA NACIONAL...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
2,04847/2019-PE,10/09/2019,09/30/2019,Decretado a...,LEY QUE MODIFICA EL ARTÍCULO 8 DE LA LEY 28301...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
3,04846/2019-PE,10/10/2019,09/27/2019,En comisión,RESOLUCIÓN LEGISLATIVA QUE MODIFICA EL ANEXO D...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
4,04845/2019-CR,10/10/2019,09/25/2019,En comisión,LEY QUE DECLARA DE INTERÉS NACIONAL Y NECESIDA...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...


In [11]:
dataGeneral["linkProy"][0]

'http://www2.congreso.gob.pe/Sicr/TraDocEstProc/CLProLey2016.nsf/641842f7e5d631bd052578e20058a231/9b8774296ca6951405258485005cba89?OpenDocument'

In [0]:
dataGeneral.to_pickle(ruta+"../data/raw/dataGeneral.pkl")

In [0]:
listSeg=[]
for link in dataGeneral["linkProy"]:
  request=urllib.request.Request(link,headers = {"User-Agent":user_agent})
  page=urllib.request.urlopen(request)
  soup=BeautifulSoup(page, "html.parser")
  aux=[]
  for n,i in enumerate(soup.find_all("tr", {"valign":"top"})[1:11]):##es del 1 al 11 porque solo se quiere solo los 10 campos, aunque hay algunos proyectos de ley que tienen mas de 10 campos
    if n==8: ### para obtener todos los nombres de los autores
      aux.append(soup.find("input",{"name":"NomCongre"}).get("value").split(","))
    else:
      aux.append(i.find_all("td")[-1].text)
  try:
    if "iniciativas" in soup.find_all("tr", {"valign":"top"})[11].text.lower():
      aux.append(soup.find_all("tr", {"valign":"top"})[11].text.split(":")[1])
    else:
      aux.append("None")
  except:
    aux.append("None")
  listSeg.append(aux) 

In [0]:
columnas=['periodo', 'legislatura', 'fechaPresentacion', 'codigo',
        'proponente', 'grupoParlamentario', 'titulo', 'sumilla',
        'autores', 'seguimiento',"inicAgrupadas"]
dataSeg=pd.DataFrame(np.asarray(listSeg),columns=columnas)
dataSeg.head()

,periodo,legislatura,fechaPresentacion,codigo,proponente,grupoParlamentario,titulo,sumilla,autores,seguimiento,inicAgrupadas
0,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04849/2019-CR,Congreso,"Frente Amplio por Justicia, Vida y Libertad","LEY DE REFORMA CONSTITUCIONAL, QUE ESTABLECE L...",Propone modificar el artículo 134 de la Const...,"[Morales Ramírez Edyson Humberto, Foronda Far...",09/10/2019 Decretado a... Constitución y Regl...,None
1,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04848/2019-PE,Poder Ejecutivo,,RESOLUCIÓN LEGISLATIVA QUE CONCEDE LA NACIONAL...,Propone conceder la nacionalidad peruana a lo...,[],09/10/2019 Decretado a... Constitución y Regl...,None
2,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04847/2019-PE,Poder Ejecutivo,,LEY QUE MODIFICA EL ARTÍCULO 8 DE LA LEY 28301...,Propone modificar el artículo 8 de la Ley 283...,[],09/10/2019 Decretado a... Constitución y Regl...,None
3,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/27/2019,04846/2019-PE,Poder Ejecutivo,,RESOLUCIÓN LEGISLATIVA QUE MODIFICA EL ANEXO D...,Propone Resolución Legislativa que modifica e...,[],"04/10/2019 Decretado a... Defensa Nacional, O...",None
4,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/25/2019,04845/2019-CR,Congreso,Contigo,LEY QUE DECLARA DE INTERÉS NACIONAL Y NECESIDA...,Propone declarar de interés nacional y necesi...,"[Violeta López Gilbert Félix, Castro Bravo J...",04/10/2019 Decretado a... Transportes y Comun...,None


In [0]:
urlsOpenDoc="http://www2.congreso.gob.pe/sicr/tradocestproc/Expvirt_2011.nsf/visbusqptramdoc1621/{}?opendocument"
urlsPdfPl="http://www.leyes.congreso.gob.pe/Documentos/2016_2021/Proyectos_de_Ley_y_de_Resoluciones_Legislativas/\S+"
listUrlsOpenDoc=[urlsOpenDoc.format(cod) for cod in dataSeg["codigo"].apply(lambda x: x.split("/")[0])]  
listUrlsPdfPl=[]
# listCodFech=list(dataSeg["codigo"].apply(lambda x : x[1:].split("/")[0])+dataSeg["fechaPresentacion"].apply(lambda x : x.split("/")[2]+x.split("/")[0]+x.split("/")[1]))
listCodFech=list(dataSeg["codigo"].apply(lambda x : x[1:].split("/")[0]))
for url,codFech in zip(listUrlsOpenDoc,listCodFech):
  try:
    request=urllib.request.Request(url,headers = {"User-Agent":user_agent})
    page=urllib.request.urlopen(request)##validar si existe el link de expediente
    soup=BeautifulSoup(page, "html.parser")  
    href=" ".join([i.get("href") for i in  soup.find_all("a")])##obtener todos los links
    aux=re.findall("\S+{}\S+".format(codFech)," ".join(re.findall(urlsPdfPl,href.replace("')",""))))[0]##solo los links de proy de ley y luego buscar por codigo
    urllib.request.urlopen(aux)###validar si el link esta disponible
    listUrlsPdfPl.append(aux)
  except:
    listUrlsPdfPl.append("None") ### no hay el link del expediente, no hay link de proy de ley  o el link del proy no esta disponible
    print(codFech)### No puede obtener la información de 9 links "4289, 2972,2249,1787,1601,1281,0753,0644,0355"
dataSeg["urlPdf"]=listUrlsPdfPl  

In [0]:
print(dataSeg.shape)
dataSeg.head()

(4849, 12)


,periodo,legislatura,fechaPresentacion,codigo,proponente,grupoParlamentario,titulo,sumilla,autores,seguimiento,inicAgrupadas,urlPdf
0,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04849/2019-CR,Congreso,"Frente Amplio por Justicia, Vida y Libertad","LEY DE REFORMA CONSTITUCIONAL, QUE ESTABLECE L...",Propone modificar el artículo 134 de la Const...,"[Morales Ramírez Edyson Humberto, Foronda Far...",09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
1,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04848/2019-PE,Poder Ejecutivo,,RESOLUCIÓN LEGISLATIVA QUE CONCEDE LA NACIONAL...,Propone conceder la nacionalidad peruana a lo...,[],09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
2,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04847/2019-PE,Poder Ejecutivo,,LEY QUE MODIFICA EL ARTÍCULO 8 DE LA LEY 28301...,Propone modificar el artículo 8 de la Ley 283...,[],09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
3,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/27/2019,04846/2019-PE,Poder Ejecutivo,,RESOLUCIÓN LEGISLATIVA QUE MODIFICA EL ANEXO D...,Propone Resolución Legislativa que modifica e...,[],"04/10/2019 Decretado a... Defensa Nacional, O...",None,http://www.leyes.congreso.gob.pe/Documentos/20...
4,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/25/2019,04845/2019-CR,Congreso,Contigo,LEY QUE DECLARA DE INTERÉS NACIONAL Y NECESIDA...,Propone declarar de interés nacional y necesi...,"[Violeta López Gilbert Félix, Castro Bravo J...",04/10/2019 Decretado a... Transportes y Comun...,None,http://www.leyes.congreso.gob.pe/Documentos/20...


In [0]:
dataSeg.to_pickle("data/raw/dataSeg.pkl")

In [12]:
index=dataGeneral["codigo"].apply(lambda x: x[1:5])
dataGeneral.set_index(index,inplace=True)
dataGeneral.index.names = ['index']
print(dataGeneral.shape)
dataGeneral.head()

(4849, 6)


,codigo,fechUlt,fechPres,estado,tituloProy,linkProy
index,,,,,,
4849,04849/2019-CR,10/09/2019,09/30/2019,Decretado a...,"LEY DE REFORMA CONSTITUCIONAL, QUE ESTABLECE L...",http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
4848,04848/2019-PE,10/10/2019,09/30/2019,En comisión,RESOLUCIÓN LEGISLATIVA QUE CONCEDE LA NACIONAL...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
4847,04847/2019-PE,10/09/2019,09/30/2019,Decretado a...,LEY QUE MODIFICA EL ARTÍCULO 8 DE LA LEY 28301...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
4846,04846/2019-PE,10/10/2019,09/27/2019,En comisión,RESOLUCIÓN LEGISLATIVA QUE MODIFICA EL ANEXO D...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...
4845,04845/2019-CR,10/10/2019,09/25/2019,En comisión,LEY QUE DECLARA DE INTERÉS NACIONAL Y NECESIDA...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...


In [13]:
index=dataSeg["codigo"].apply(lambda x: x[1:5])
dataSeg.set_index(index,inplace=True)
dataSeg.index.names = ['index']
print(dataSeg.shape)
dataSeg.head()

(4849, 12)


,periodo,legislatura,fechaPresentacion,codigo,proponente,grupoParlamentario,titulo,sumilla,autores,seguimiento,inicAgrupadas,urlPdf
index,,,,,,,,,,,,
4849,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04849/2019-CR,Congreso,"Frente Amplio por Justicia, Vida y Libertad","LEY DE REFORMA CONSTITUCIONAL, QUE ESTABLECE L...",Propone modificar el artículo 134 de la Const...,"[Morales Ramírez Edyson Humberto, Foronda Far...",09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
4848,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04848/2019-PE,Poder Ejecutivo,,RESOLUCIÓN LEGISLATIVA QUE CONCEDE LA NACIONAL...,Propone conceder la nacionalidad peruana a lo...,[],09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
4847,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/30/2019,04847/2019-PE,Poder Ejecutivo,,LEY QUE MODIFICA EL ARTÍCULO 8 DE LA LEY 28301...,Propone modificar el artículo 8 de la Ley 283...,[],09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
4846,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/27/2019,04846/2019-PE,Poder Ejecutivo,,RESOLUCIÓN LEGISLATIVA QUE MODIFICA EL ANEXO D...,Propone Resolución Legislativa que modifica e...,[],"04/10/2019 Decretado a... Defensa Nacional, O...",None,http://www.leyes.congreso.gob.pe/Documentos/20...
4845,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,09/25/2019,04845/2019-CR,Congreso,Contigo,LEY QUE DECLARA DE INTERÉS NACIONAL Y NECESIDA...,Propone declarar de interés nacional y necesi...,"[Violeta López Gilbert Félix, Castro Bravo J...",04/10/2019 Decretado a... Transportes y Comun...,None,http://www.leyes.congreso.gob.pe/Documentos/20...


In [14]:
dataMetadata=pd.merge(dataGeneral,dataSeg.drop(["fechaPresentacion","titulo","codigo"],axis=1),on="index")
print(dataMetadata.shape)
dataMetadata.head()

(4849, 15)


,codigo,fechUlt,fechPres,estado,tituloProy,linkProy,periodo,legislatura,proponente,grupoParlamentario,sumilla,autores,seguimiento,inicAgrupadas,urlPdf
index,,,,,,,,,,,,,,,
4849,04849/2019-CR,10/09/2019,09/30/2019,Decretado a...,"LEY DE REFORMA CONSTITUCIONAL, QUE ESTABLECE L...",http://www2.congreso.gob.pe/Sicr/TraDocEstProc...,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,Congreso,"Frente Amplio por Justicia, Vida y Libertad",Propone modificar el artículo 134 de la Const...,"[Morales Ramírez Edyson Humberto, Foronda Far...",09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
4848,04848/2019-PE,10/10/2019,09/30/2019,En comisión,RESOLUCIÓN LEGISLATIVA QUE CONCEDE LA NACIONAL...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,Poder Ejecutivo,,Propone conceder la nacionalidad peruana a lo...,[],09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
4847,04847/2019-PE,10/09/2019,09/30/2019,Decretado a...,LEY QUE MODIFICA EL ARTÍCULO 8 DE LA LEY 28301...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,Poder Ejecutivo,,Propone modificar el artículo 8 de la Ley 283...,[],09/10/2019 Decretado a... Constitución y Regl...,None,http://www.leyes.congreso.gob.pe/Documentos/20...
4846,04846/2019-PE,10/10/2019,09/27/2019,En comisión,RESOLUCIÓN LEGISLATIVA QUE MODIFICA EL ANEXO D...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,Poder Ejecutivo,,Propone Resolución Legislativa que modifica e...,[],"04/10/2019 Decretado a... Defensa Nacional, O...",None,http://www.leyes.congreso.gob.pe/Documentos/20...
4845,04845/2019-CR,10/10/2019,09/25/2019,En comisión,LEY QUE DECLARA DE INTERÉS NACIONAL Y NECESIDA...,http://www2.congreso.gob.pe/Sicr/TraDocEstProc...,Periodo de Gobierno 2016 - 2021.,Primera Legislatura Ordinaria 2019,Congreso,Contigo,Propone declarar de interés nacional y necesi...,"[Violeta López Gilbert Félix, Castro Bravo J...",04/10/2019 Decretado a... Transportes y Comun...,None,http://www.leyes.congreso.gob.pe/Documentos/20...


In [0]:
dataMetadata.to_pickle("../data/raw/dataMetadata.pkl")

### Obtener los pdfs de los proyectos de ley

In [0]:
listPlCod=list(dataSeg["codigo"].apply(lambda x : x.split("/")[0])+dataSeg["fechaPresentacion"].apply(lambda x : x.split("/")[2]+x.split("/")[0]+x.split("/")[1]))
for n,(url,plCod) in enumerate(zip(dataSeg["urlPdf"],listPlCod)):
  try:  
    myfile = requests.get(url)##PDF
    open(ruta+"../data/raw/pdf/PL{}.pdf".format(plCod), 'wb').write(myfile.content)
  except:
    print(n)## esto es cuando no se encuentra el link 